In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

In [2]:
def loadMovieNames():
    movie_names = {}
    with open('/Users/ayusman/migrate/hadooop/hadoop-basics/ml-100k/u.item', encoding = "ISO-8859-1") as f:
        for line in f:
            fields = line.split('|')
            movie_names[int(fields[0])] = fields[1]
    
    return  movie_names

In [3]:
def parseInput(line):
    fields = line.split()
    return (int (fields[1]), float (fields[2]), 1.0)

In [4]:
# setting spark configuration file and spark context
conf = SparkConf().setAppName("worst movies")
sc = SparkContext(conf=conf)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/27 14:21:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/01/27 14:21:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
# getting movie names
movieNames = loadMovieNames()

In [6]:
lines = sc.textFile('/Users/ayusman/migrate/hadooop/hadoop-basics/ml-100k/u.data')

In [7]:
# convert to (movieID, (rating,1.0)) which is and RDD
movieRatings = lines.map(parseInput)

In [74]:
movieRatings.sortBy(lambda x:x[0])
movieRatings.take(3)

[(242, 3.0, 1.0), (302, 3.0, 1.0), (377, 1.0, 1.0)]

In [78]:
# Reduce to (movieID, (sumofRating, toalRating))
ratingTotalCounts = movieRatings.reduceByKey(lambda movie1, movie2: (movie1[0]+movie2[0], (movie1[1]+movie2[1])))
ratingTotalCounts.take(2)

22/01/27 16:31:01 ERROR Executor: Exception in task 0.0 in stage 85.0 (TID 91)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 619, in main
    process()
  File "/opt/homebrew/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 609, in process
    out_iter = func(split_index, iterator)
  File "/opt/homebrew/lib/python3.9/site-packages/pyspark/rdd.py", line 2918, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/homebrew/lib/python3.9/site-packages/pyspark/rdd.py", line 2918, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/homebrew/lib/python3.9/site-packages/pyspark/rdd.py", line 417, in func
    return f(iterator)
  File "/opt/homebrew/lib/python3.9/site-packages/pyspark/rdd.py", line 2146, in combineLocally
    merger.mergeValues(iterator)
  File "/opt/h

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 85.0 failed 1 times, most recent failure: Lost task 0.0 in stage 85.0 (TID 91) (192.168.1.236 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 619, in main
    process()
  File "/opt/homebrew/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 609, in process
    out_iter = func(split_index, iterator)
  File "/opt/homebrew/lib/python3.9/site-packages/pyspark/rdd.py", line 2918, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/homebrew/lib/python3.9/site-packages/pyspark/rdd.py", line 2918, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/homebrew/lib/python3.9/si

In [61]:
ratingTotalCounts

PythonRDD[164] at RDD at PythonRDD.scala:53

In [71]:
# Map tp movieID and average rating
averageRating = ratingTotalCounts.mapValues(lambda totalAndCount: totalAndCount[0]/totalAndCount[1])
type(averageRating)
averageRating.take(2)

22/01/27 16:27:13 ERROR Executor: Exception in task 1.0 in stage 78.0 (TID 83)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 619, in main
    process()
  File "/opt/homebrew/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 609, in process
    out_iter = func(split_index, iterator)
  File "/opt/homebrew/lib/python3.9/site-packages/pyspark/rdd.py", line 2918, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/homebrew/lib/python3.9/site-packages/pyspark/rdd.py", line 2918, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/homebrew/lib/python3.9/site-packages/pyspark/rdd.py", line 417, in func
    return f(iterator)
  File "/opt/homebrew/lib/python3.9/site-packages/pyspark/rdd.py", line 2146, in combineLocally
    merger.mergeValues(iterator)
  File "/opt/h

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 78.0 failed 1 times, most recent failure: Lost task 1.0 in stage 78.0 (TID 83) (192.168.1.236 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 619, in main
    process()
  File "/opt/homebrew/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 609, in process
    out_iter = func(split_index, iterator)
  File "/opt/homebrew/lib/python3.9/site-packages/pyspark/rdd.py", line 2918, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/homebrew/lib/python3.9/site-packages/pyspark/rdd.py", line 2918, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/homebrew/lib/python3.9/si

In [49]:
# sort by average rating
#sortedAverageRating = averageRating.sortBy(lambda x: x[0])